this notebook is used to study mass, momentum, and energy fluxes. code plots mostly the mollview projections of many different kinds of space and temporal evolutions.

In [ ]:
from postprocessing import *
import healpy as hp
import glob
import os
from natsort import natsorted

In [ ]:
mpl.rcParams['figure.dpi']= 200

In [ ]:
def mass_flux(directory, i_file, projection='radial', weights='full'):
    """
    Calculate mass flux for each cell for given snapshot and different projection types.
    Input: directory  (a directory where outputs are stored)
           i_file     (snapshot number)
           projection (projection type, can be 'radial', 'transverse', or 'total')
           weights    ('full' if all particles are considered and 'jet-tracer' if only jet's contribution is needed)
    """
    filename = "snap_%03d.hdf5" % (i_file)
    snap_data = h5py.File(directory + filename, "r")
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    v_r = (vx * (x - 500) + vy * (y - 500) + vz * (z - 500)) / np.sqrt(x ** 2 + y ** 2 + z ** 2)
    if projection == 'radial':
        j = snap_data['PartType0/Density'] * v_r * unit_density * unit_velocity
    elif projection == 'transverse':
        v_t = np.sqrt(vx ** 2 + vy ** 2 + vz ** 2 - v_r ** 2)
        j = snap_data['PartType0/Density'] * v_t * unit_density * unit_velocity
    elif projection == 'total':
        v = np.sqrt(vx ** 2 + vy ** 2 + vz ** 2)
        j = snap_data['PartType0/Density'] * v * unit_density * unit_velocity
    else:
        raise ValueError("Wrong projection type. Expected 'radial', 'transverse', or 'total'. ")
        
    if weights == 'jet-tracer':
        j *= snap_data['PartType0/Jet_Tracer']
    return j, x - 500, y - 500, z - 500, get_time_from_snap(snap_data) * unit_time_in_megayr

def mass_flux_shell(directory, i_file, projection, radius, dr=20):
    """
    Calculate mass flux for each cell within a certain radius for given snapshot and different projection types.
    Input: directory  (a directory where outputs are stored)
           i_file     (snapshot number)
           projection (projection type, can be 'radial', 'transverse', or 'total')
           radius     (at what radius the flux is calculated)
           dr         (the shell width around radius)
    """
    j_all, x_all, y_all, z_all, time = mass_flux(directory, i_file, projection=projection)
    
    radius_range = np.linspace(40, 500, 31)
    NSIDE = 31
    NPIX = hp.nside2npix(NSIDE)
    theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX)) # return colatitude and longtitude in radian
    vec = hp.ang2vec(theta, phi)  # return unit 3D position vector

    vec_scaled = vec * radius
    mask = ((np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) < radius + dr) & 
            (np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) > radius - dr))
    j, x, y, z = j_all[mask], x_all[mask], y_all[mask], z_all[mask]
    j_shell = []

    for vector in vec_scaled:
        distance = (vector[0] - x) ** 2 + (vector[1] - y) ** 2 + (vector[2] - z) ** 2
        j_shell.append(j[distance.argmin()])
    return j_shell

def total_energy(directory, i_file):
    """
    Calculate energy flux for each cell for given snapshot
    Input: directory  (a directory where outputs are stored)
           i_file     (snapshot number)
    """
    filename = "snap_%03d.hdf5" % (i_file)
    snap_data = h5py.File(directory + filename, "r")
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    v_square = vx ** 2 + vy ** 2 + vz ** 2
    E = snap_data['PartType0/Masses'] * v_square / 2 + snap_data['PartType0/InternalEnergy']
    return E, x - 500, y - 500, z - 500, get_time_from_snap(snap_data) * unit_time_in_megayr

In [ ]:
density = '30'
mach    = '8'
jet     = '43'
start   = 'early'

In [ ]:
simulation_directory = str(f'/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d{density}_m{mach}/jet{jet}_{start}')
#simulation_directory = str(f'/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d{density}_m{mach}/turb')

output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"


## mass flux propogation in r

$j = \rho \cdot v_r $

In [ ]:
i_file = 5
projection = 'radial'

j_all, x_all, y_all, z_all, time = mass_flux(output_directory, i_file, proj=projection)

In [ ]:
radius_range = np.linspace(40, 500, 31)
NSIDE = 31
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX)) # return colatitude and longtitude in radian
vec = hp.ang2vec(theta,phi)  # return unit 3D position vector

In [ ]:
for radius in radius_range:
    print(radius)
    vec_scaled = vec * radius
    temperatures = get_temp(output_directory + "snap_%03d.hdf5" % (i_file), 5/3)
    mask = ((np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) < radius + 20) & 
            (np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) > radius - 20) &
            (temperatures > 10 ** 4.5))
    j, x, y, z = j_all[mask], x_all[mask], y_all[mask], z_all[mask]
    j_shell = []

    for vector in vec_scaled:
        distance = (vector[0] - x) ** 2 + (vector[1] - y) ** 2 + (vector[2] - z) ** 2
        try: 
            j_shell.append(j[distance.argmin()])
        except: j_shell.append(np.nan)
    
    if projection == 'radial':
        hp.mollview(np.array(j_shell), title=fr"Mass flux in $r \in$ [{np.round(radius,0) - 20}; {np.round(radius,0) + 20}] pc", 
        min=-1e-16, max=1e-16, unit=r"mass flux [g cm$^{-2}$ s$^{-1}$]", cmap='coolwarm', rot=(90,0,0))
    else:
        hp.mollview(np.array(j_shell), title=fr"Mass flux in $r \in$ [{np.round(radius,0) - 20}; {np.round(radius,0) + 20}] pc", 
            min=1e-17, max=5e-14, unit="mass flux in [g cm$^{-2}$ s$^{-1}$]", norm='log', cmap='magma', rot=(90,0,0))
        
    hp.projscatter((np.pi/2, 0), s=30, c='red')
    hp.projscatter((np.pi/2, np.pi), s=30, c='blue')
    
    hp.graticule()
    plt.savefig(figures_directory + f'massflux_mollview_{projection}_hot_{i_file}_{np.round(radius, 0)}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + f'/massflux_mollview_{projection}_hot_{i_file}_*.png'
ofilename = figures_directory + f'/massflux_mollview_{projection}_hot_{i_file}-jet.gif'
imgs = natsorted(glob.glob(ifilename))

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=160, loop=0)

## mass flux propogation in t

In [ ]:
radius = 400

In [ ]:

for i_file in range(15):
    j_r   = mass_flux_shell(output_directory, i_file, 'radial', radius)
    j_t   = mass_flux_shell(output_directory, i_file, 'transverse', radius)
    j_tot = mass_flux_shell(output_directory, i_file, 'total', radius)
    
    fig, ax = plt.subplots(2, 2, figsize=(11, 6.1))

    fig.tight_layout(w_pad=7.0, h_pad=3.0)

    fig.suptitle(fr"Mass flux in $r \in$ [{np.round(radius,0) - 20}; {np.round(radius,0) + 20}] pc", 
                 y=1.05, fontsize=15)


    plt.axes(ax[0][0])
    hp.mollview(np.array(j_r), title=fr"$j_r$", rot=(90,0,0),
            min=-5e2, max=5e2, unit=r"mass flux $\left[\frac{M_\odot km}{pc^3 s}\right]$",
                cmap='coolwarm', hold=True)

    plt.axes(ax[0][1])
    hp.mollview(np.array(j_t), title=fr"$j_t$", rot=(90,0,0),
            min=0, max=7e2,  unit=r"mass flux $\left[\frac{M_\odot km}{pc^3 s}\right]$",
                cmap='Reds', hold=True)

    plt.axes(ax[1][0])
    hp.mollview(np.array(j_tot), title=r"$j_{tot}$", rot=(90,0,0),
            min=0, max=7e2, unit=r"mass flux $\left[\frac{M_\odot km}{pc^3 s}\right]$",
                cmap='Reds', hold=True)

    plt.axes(ax[1][1])
    hp.mollview(np.array(np.array(np.abs(j_r)) / (np.array(j_t) + 1e-5)), title=fr"$|j_r| \ / \ j_t$", rot=(90,0,0),
            min=0, max=1, unit=r"mass flux $\left[\frac{M_\odot km}{pc^3 s}\right]$", 
                cmap='Reds', hold=True)

    hp.projscatter((np.pi/2, 0), s=30, c='red')
    hp.projscatter((np.pi/2, np.pi), s=30, c='blue')
    
    f = plt.gcf().get_children()

    for i in [2, 4, 6, 8]:
        CbAx = f[i]

        unit_text_obj = CbAx.get_children()[1]
        unit_text_obj.set_fontsize(12)
        unit_text_obj.set_position((0.5, -2.2))

    hp.graticule()

    plt.savefig(figures_directory + f'massflux_mollview_allprojections_{i_file}_{np.round(radius, 0)}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
radius = 400
projection = 'radial'

In [ ]:
NSIDE = 31
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX)) # return colatitude and longtitude in radian
vec = hp.ang2vec(theta,phi) * radius

In [ ]:
for i_file in range(12):
    j_all, x_all, y_all, z_all, time = mass_flux(output_directory, i_file)
    mask = ((np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) < radius + 10) & 
            (np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) > radius - 10))
    j, x, y, z = j_all[mask], x_all[mask], y_all[mask], z_all[mask]
    j_shell = []

    for vector in vec:
        distance = (vector[0] - x) ** 2 + (vector[1] - y) ** 2 + (vector[2] - z) ** 2
        j_shell.append(j[distance.argmin()])

    hp.mollview(np.array(j_shell), title=fr"Mass flux in $r \in$ [{np.round(radius,0) - 10}; {np.round(radius,0) + 10}] pc", 
            min=200, max=5e5,
            unit="mass flux in cgs", norm='log', cmap='magma')
    hp.graticule()
    plt.savefig(figures_directory + f'massflux_mollview_{projection}_{i_file}_{np.round(radius, 0)}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
# !ls /n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d100_m8/jet42_early/output/figures/

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

# ifilename = figures_directory + f'/massflux_mollview_{projection}*{radius}.png'
# ofilename = figures_directory + f'/massflux_mollview_{projection}-time-jet.gif'

ifilename = figures_directory + f'/massflux_mollview_allprojections*{radius}.png'
ofilename = figures_directory + f'/massflux_mollview_allprojections-time-jet.gif'


imgs = natsorted(glob.glob(ifilename))

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=160, loop=0)

## total energy flux

In [ ]:
i_file = 8
E_all, x_all, y_all, z_all, time = total_energy(output_directory, i_file)

In [ ]:
radius_range = np.linspace(40, 500, 31)
NSIDE = 31
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX)) # return colatitude and longtitude in radian
vec = hp.ang2vec(theta,phi)  # return unit 3D position vector

In [ ]:
for radius in radius_range:
    print(radius)
    vec_scaled = vec * radius
    mask = ((np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) < radius + 20) & 
            (np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) > radius - 20))
    E, x, y, z = E_all[mask], x_all[mask], y_all[mask], z_all[mask]
    E_shell = []

    for vector in vec_scaled:
        distance = (vector[0] - x) ** 2 + (vector[1] - y) ** 2 + (vector[2] - z) ** 2
        E_shell.append(E[distance.argmin()])
    
    hp.mollview(np.array(E_shell), title=fr"Total energy in $r \in$ [{np.round(radius,0) - 20}; {np.round(radius,0) + 20}] pc", 
            min=1e5, max=1e8,
            unit="total energy in cgs", norm='log', cmap='magma')
    hp.graticule()
    plt.savefig(figures_directory + f'totalenergy_mollview_{i_file}_{np.round(radius, 0)}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + '/totalenergy_mollview_8_*.png'
ofilename = figures_directory + '/totalenergy_mollview_8-jet.gif'
imgs = natsorted(glob.glob(ifilename))

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=160, loop=0)

## mass rate outflow 

In [ ]:
simulation_directory = str('/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d100_m8/jet42_early')
output_directory = simulation_directory+"/output/"
figures_directory = simulation_directory + "/output/figures/"
i_file = 7

$\dot{m} = \frac{dm}{dt} = \rho \dot{V} = j_r A$

or when a curved surface:

$\dot{m}(R) = \iint \vec{j} \cdot \vec{d A} = 4 \pi R^2 j_r$

In [ ]:
filename = "snap_%03d.hdf5" % (i_file)
snap_data = h5py.File(output_directory + filename, "r")
x, y, z = snap_data['PartType0/Coordinates'][:].T
vx, vy, vz = snap_data['PartType0/Velocities'][:].T
v_r = (vx * (x - 500) + vy * (y - 500) + vz * (z - 500)) / np.sqrt(x ** 2 + y ** 2 + z ** 2)

outflow = (snap_data['PartType0/Density']                              #M_sun / kpc^3
           * v_r * 4 * np.pi  / 3.24078e-17  *                         #km/s -> kpc/s
           ((x - 500) ** 2 + (y - 500) ** 2 + (z - 500) ** 2)) /1e6    #pc^2 -> kpc^2
radius=400

In [ ]:
def calculate_outflow(phi, theta, radius):
    x_dir = radius * np.sin(phi) * np.cos(theta)
    y_dir = radius * np.sin(phi) * np.sin(theta)
    z_dir = radius * np.cos(phi)
    distance = (x_dir - x + 500) ** 2 + (y_dir - y + 500) ** 2 + (z_dir - z + 500) ** 2
    return outflow[distance.argmin()]

In [ ]:
import numpy as np
from scipy import integrate

In [ ]:
f = lambda theta, phi: np.sin(phi) * calculate_outflow(phi, theta, radius)
integrate.dblquad(f, 0, np.pi, 0, 2 * np.pi, epsrel = 1e-4)[0]

## mass flux distribution 

In [ ]:
bins = np.linspace(0, 2e-12, 100)

In [ ]:
i_file = 1
projection = 'radial'

filename = "snap_%03d.hdf5" % (i_file)
snap_data = h5py.File(output_directory + filename, "r")

j_all, x_all, y_all, z_all, time = mass_flux(output_directory, i_file, proj=projection)
temperatures = get_temp(output_directory + filename, 5/3)
masses = snap_data['PartType0/Masses'][:]

mask = (temperatures > 10 ** 4.5)

In [ ]:
plt.hist(np.abs(j_all), bins=bins, log=True, weights=masses)
plt.hist(np.abs(j_all[mask]), bins=bins, log=True, weights=masses[mask])

plt.title(fr'$j_r$ distribution at t = {np.round(get_time_from_snap(snap_data) * unit_time_in_megayr, 2)} Myr')
plt.xlim(0, 2e-12)